In [1]:
# coding: utf-8
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import os
current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)

import sys
sys.path.append(current_dir)

import weth_api as wa


In [3]:
settings = wa.read_yaml_config('config.yaml', section='api.weatherapi.com')
base_url = settings['url']
method = settings['method']
api_key = settings['api_key']

gwd = wa.get_weth_data(api_key, base_url, method, '36.757594, 31.45268', 7)
df = wa.load_weth_data_to_df(gwd)

In [4]:
df.sample(3)

,time,is_day,temp_c,feelslike_c,windchill_c,heatindex_c,condition,wind_kph,wind_dir,pressure_hg,precip_mm,humidity,cloud,will_it_rain,will_it_snow,chance_of_rain,chance_of_snow,uv,place,region,country,lat,lon
11,2024-07-25 11:00,1,29.9,34.8,29.9,34.8,Sunny,17.6,S,750.811745,0.0,70,9,0,0,0,0,7.0,Manavgat,Antalya,Turkey,36.76,31.45
2,2024-07-23 02:00,0,27.7,30.8,27.7,30.8,Clear,4.3,NNE,751.561806,0.0,73,3,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45
0,2024-07-23 00:00,0,27.8,31.6,27.8,31.6,Clear,4.3,E,752.311868,0.0,77,3,0,0,0,0,0.0,Manavgat,Antalya,Turkey,36.76,31.45
